In [ ]:
pip install selenium

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects, RequestException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
import concurrent.futures

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def scrape_page(url, type):
    data = {}
    try:

        # response = requests.get(url)
        # if response.status_code != 200:
        #     print(f"Failed to retrieve the page. Status code: {response.status_code}")
        #     return None

        # Proceed with Selenium if the status is OK
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-extensions')
        options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
        # Add any required options here
        browser = webdriver.Chrome(options=options)

        browser.get(url)

        # Wait for the page to load
        wait = WebDriverWait(browser, 10)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        # Continue with your scraping logic

        # Scroll down incrementally
        # for i in range(10):  # Scroll down 10 times
        #     browser.execute_script("window.scrollBy(0, 800);")
        #     time.sleep(0.8)  # Adjust sleep time if needed

        # time.sleep(10)
        # Get the rendered HTML
        html = browser.page_source

        # Initiate data object
        data = {'source': url, 'source_name': 'ose', 'Listing Type': type}

        # Parse the HTML content
        soup = BeautifulSoup(html, 'html.parser')

        body = soup.find(id='page-container')

        price = body.find(class_='price_box')
        price_list = price.text.strip().split('Per')
        data['price'] = price_list[0].strip()

        desc0 = soup.find(id='desc-0')
        description = desc0.find(id='show_content_1')
        data['description'] = description.text.strip()

        desc1 = soup.find(id='desc-1')
        description = desc1.find(id='show_content_1')
        divs = description.find_all('div')
        for div in divs:
            label = div.find(name='label')
            details = div.find(name='span')
            data[label.text.strip(' :')] = details.text.strip()

        desc2 = soup.find(id='desc-2')
        description = desc2.find(id='show_content_2')
        divs = description.find_all('div')
        location = ''
        for div in divs:
            # label = div.find('label')
            details = div.find('span')
            if location == '':
                location = details.text.strip()
            else:
                location = location + ', ' + details.text.strip()
        data['location'] = location

        # Get the location string
        location = soup.select('a[href^="https://maps.google.com/maps?ll="]')
        if location:
            data['img_src'] = location[0]['href']
            img = data['img_src'].removeprefix("https://maps.google.com/maps?ll=")
            coordinate = img.removesuffix("&z=10&t=m&hl=en-US&gl=US&mapclient=apiv3")
            coordinate = coordinate.split(',')
            x = coordinate[0]
            y = coordinate[1]
            data['X'] = x
            data['Y'] = y

        return data
    except TimeoutException:
        print("Timed out waiting for page to load")
    except ConnectionError as e:
        print(f"Connection error occurred: {e}")
    except Timeout as e:
        print(f"Timeout error occurred: {e}")
    except TooManyRedirects as e:
        print(f"Too many redirects: {e}")
    except RequestException as e:
        print(f"An error occurred RequestException: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        browser.quit()  # Close the browser even if an exception occurs

    return None

In [ ]:
def scraping_data(path, file_name, type):
    # Load the CSV file into a DataFrame
    urls = pd.read_csv(path + file_name)
    print(len(urls))
    all_data = []
    failed_url = []
    index = 1
    def process_url(url, type):
        print(f"Processing {url}")
        return scrape_page(url, type)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Map the process_url function to the list of URLs
        future_to_url = {executor.submit(process_url, url['URL'], type): url for index, url in urls.iterrows()}

        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                page_data = future.result()
                all_data.append(page_data)
                index += 1
                print(index)
            except Exception as exc:
                print(f"{url['URL']} generated an exception: {exc}")
                failed_url.append(url['URL'])

    # Filter out None values
    filtered_data = [item for item in all_data if item is not None]

    # Create DataFrame
    df = pd.DataFrame(filtered_data)

    # Rename columns
    df = df.rename(columns={
        'Listing ID': 'Reference Id',
        'Square Meter': 'Size (m²)',
        'Price Type': 'Rental duration'
    })

    #remove unused column
    df = df.drop('Price')

    csv_file_path = f'{path}data_{file_name}'
    df.to_csv(csv_file_path, index=False)
    return all_data, failed_url

In [ ]:
# Replace the path with your file's path
# file_path_rent = '/content/drive/MyDrive/M2 final project/realestate website/'
file_path_sale = '/content/drive/MyDrive/M2 final project/ose properties/'
d,f = scraping_data(file_path_sale, 'urls_to_be_scraped_rent.csv')
# scraping_data(file_path_rent, 'urls_to_be_scraped_rent.csv')

153
Processing https://www.ose-properties.com/properties/480-Office-Achrafieh-Beirut-Lebanon-1-Bathroom-USD650/
Processing https://www.ose-properties.com/properties/527-Apartment-Achrafieh-Beirut-Lebanon-2-Rooms-2-Bedrooms-2-Bathrooms-USD1-100/
Processing https://www.ose-properties.com/properties/595-Office-Badaro-Beirut-Lebanon-2-Bathrooms-USD2-000/
Processing https://www.ose-properties.com/properties/678-Chalet-Tabarja-Keserwen-Lebanon-1-Room-1-Bedroom-1-Bathroom-USD1-700/
Processing https://www.ose-properties.com/properties/528-Apartment-Achrafieh-Beirut-Lebanon-2-Rooms-2-Bedrooms-1-Bathroom-USD1-000/
Processing https://www.ose-properties.com/properties/426-Apartment-Wadi-Shahrour-Assoufla-Baabda-Lebanon-2-Rooms-2-Bedrooms-2-Bathrooms-USD550/
Processing https://www.ose-properties.com/properties/572-Apartment-Gemmayzeh-Beirut-Lebanon-3-Rooms-3-Bedrooms-3-Bathrooms-USD3-000/
3
Processing https://www.ose-properties.com/properties/284-Apartment-Antelias-Metn-Lebanon-3-Rooms-3-Bedrooms-3